In [10]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [12]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

In [13]:
df_torax = df_procesada.query("COD_HOSPITAL == 112103")

## 1. Análisis Sociodemográfico de los pacientes

En este apartado se quiere responder las siguientes preguntas:

- ¿Cuál es la **edad** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es el **sexo** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es la **previsión** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es la **letra de FONASA** de los pacientes que se realizan los procedimientos en el hospital del Tórax?

Para responder estas preguntas se utilizará la base de datos de GRD Público entre 2019 y 2021.

In [15]:
columnas_sociodemográficas = [
    "PREVISION",
    "SEXO",
    "RANGO_ETARIO",
    "Región y País",
    "Comuna, Región y País",
    "SERVICIO_SALUD",
]

df_procesada_formato_largo = pd.melt(
    df_torax,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "CIP_ENCRIPTADO"] + columnas_sociodemográficas,
    value_vars=vars_procedimientos.keys(),
    value_name="procedimiento",
)

In [16]:
agrupacion_estatica = ["ANIO_EGRESO", "Región y País", "Comuna, Región y País"]
agrupacion_variable = ["PREVISION", "SEXO", "RANGO_ETARIO", "SERVICIO_SALUD"]
variable_a_contar = "procedimiento"

In [17]:
resultados_sociodemograficos = build_features.obtener_desglose_sociodemografico(
    df_procesada_formato_largo, agrupacion_estatica, agrupacion_variable, variable_a_contar
)

In [18]:
ruta_a_guardar = "../data/interim/sociodemografico/desglose_sociodemografico_proced.xlsx"

with pd.ExcelWriter(ruta_a_guardar) as file:
    for nombre_hoja, df_a_guardar in resultados_sociodemograficos.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja, index=False)

A modo de ejemplo, se quiere saber el desglose sociodemográfico de las Angioplastías (00.66) en 2019
para el Hospital del Tórax

In [19]:
resultados_sociodemograficos["global"].query("procedimiento == '00.66' and ANIO_EGRESO == 2019.0")

,ANIO_EGRESO,Región y País,"Comuna, Región y País",procedimiento,conteo_global,llave_id
546,2019.0,"Región Metropolitana de Santiago, Chile","Alhué, Región Metropolitana de Santiago, Chile",00.66,3,"2019.0-Región Metropolitana de Santiago, Chile..."
673,2019.0,"Región Metropolitana de Santiago, Chile","Buin, Región Metropolitana de Santiago, Chile",00.66,1,"2019.0-Región Metropolitana de Santiago, Chile..."
839,2019.0,"Región Metropolitana de Santiago, Chile","Cerrillos, Región Metropolitana de Santiago, C...",00.66,1,"2019.0-Región Metropolitana de Santiago, Chile..."
936,2019.0,"Región Metropolitana de Santiago, Chile","Cerro Navia, Región Metropolitana de Santiago,...",00.66,2,"2019.0-Región Metropolitana de Santiago, Chile..."
1044,2019.0,"Región Metropolitana de Santiago, Chile","Colina, Región Metropolitana de Santiago, Chile",00.66,1,"2019.0-Región Metropolitana de Santiago, Chile..."
...,...,...,...,...,...,...
13940,2019.0,Región del Libertador General Bernardo O'Higgi...,"Santa Cruz, Región del Libertador General Bern...",00.66,2,2019.0-Región del Libertador General Bernardo ...
14067,2019.0,"Región del Maule, Chile","Constitución, Región del Maule, Chile",00.66,2,"2019.0-Región del Maule, Chile-Constitución, R..."
14305,2019.0,"Región del Maule, Chile","Parral, Región del Maule, Chile",00.66,1,"2019.0-Región del Maule, Chile-Parral, Región ..."
14395,2019.0,"Región del Maule, Chile","Río Claro, Región del Maule, Chile",00.66,1,"2019.0-Región del Maule, Chile-Río Claro, Regi..."


Los resultados muestran el desglose por región y comuna para las angioplastias.

Además se quiere saber si efectivamente la base original posee esa cantidad de angioplastias.

In [20]:
consulta_inicial = df_procesada.query("ANIO_EGRESO == 2019.0 and COD_HOSPITAL == 112103")

filas_con_procedimiento = (
    consulta_inicial[vars_procedimientos.keys()]
    .apply(lambda x: x.str.contains("00.66"))
    .sum(axis=1)
    .astype(bool)
)

control_angioplastias = consulta_inicial[filas_con_procedimiento]
display(control_angioplastias)

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA,RANGO_ETARIO,Código Comuna,Código Región,Nombre Región,Región y País,"Comuna, Región y País"
9443,112103,317615.0,HOMBRE,1949-05-28,NINGUNO,SANTIAGO,ÑUÑOA,CHILE,FONASA B,METROPOLITANO ORIENTE,...,0.0,2019.0,2.0,2019-2,"(60, 70]",13120.0,13.0,Metropolitana de Santiago,"Región Metropolitana de Santiago, Chile","Ñuñoa, Región Metropolitana de Santiago, Chile"
16217,112103,1137216.0,MUJER,1955-10-15,NINGUNO,SANTIAGO,PEÑALOLÉN,CHILE,FONASA B,METROPOLITANO ORIENTE,...,1.0,2019.0,2.0,2019-2,"(60, 70]",13122.0,13.0,Metropolitana de Santiago,"Región Metropolitana de Santiago, Chile","Peñalolén, Región Metropolitana de Santiago, C..."
16229,112103,665546.0,HOMBRE,1949-08-23,NINGUNO,ELQUI,COQUIMBO,CHILE,FONASA D,COQUIMBO,...,1.0,2019.0,2.0,2019-2,"(60, 70]",4102.0,4.0,de Coquimbo,"Región de Coquimbo, Chile","Coquimbo, Región de Coquimbo, Chile"
16233,112103,1356849.0,HOMBRE,1955-07-16,NINGUNO,SANTIAGO,RECOLETA,CHILE,FONASA D,METROPOLITANO NORTE,...,3.0,2019.0,3.0,2019-3,"(60, 70]",13127.0,13.0,Metropolitana de Santiago,"Región Metropolitana de Santiago, Chile","Recoleta, Región Metropolitana de Santiago, Chile"
16235,112103,900579.0,MUJER,1946-07-13,NINGUNO,ELQUI,COQUIMBO,CHILE,FONASA B,COQUIMBO,...,1.0,2019.0,3.0,2019-3,"(70, 80]",4102.0,4.0,de Coquimbo,"Región de Coquimbo, Chile","Coquimbo, Región de Coquimbo, Chile"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138506,112103,984038.0,MUJER,1960-04-23,NINGUNO,SANTIAGO,LAS CONDES,CHILE,FONASA D,METROPOLITANO ORIENTE,...,1.0,2019.0,12.0,2019-12,"(50, 60]",13114.0,13.0,Metropolitana de Santiago,"Región Metropolitana de Santiago, Chile","Las Condes, Región Metropolitana de Santiago, ..."
1139267,112103,111840.0,HOMBRE,1957-09-18,NINGUNO,SANTIAGO,MACUL,CHILE,FONASA D,METROPOLITANO ORIENTE,...,0.0,2019.0,3.0,2019-3,"(60, 70]",13118.0,13.0,Metropolitana de Santiago,"Región Metropolitana de Santiago, Chile","Macul, Región Metropolitana de Santiago, Chile"
1139269,112103,671479.0,HOMBRE,1971-07-08,NINGUNO,SANTIAGO,ÑUÑOA,CHILE,FONASA D,METROPOLITANO ORIENTE,...,0.0,2019.0,3.0,2019-3,"(40, 50]",13120.0,13.0,Metropolitana de Santiago,"Región Metropolitana de Santiago, Chile","Ñuñoa, Región Metropolitana de Santiago, Chile"
1141638,112103,101780.0,HOMBRE,1937-01-21,NINGUNO,SANTIAGO,ÑUÑOA,CHILE,FONASA D,METROPOLITANO ORIENTE,...,2.0,2019.0,2.0,2019-2,"(80, 90]",13120.0,13.0,Metropolitana de Santiago,"Región Metropolitana de Santiago, Chile","Ñuñoa, Región Metropolitana de Santiago, Chile"


La base original muestra que existen 830 egresos que contienen al menos 1 procedimiento 00.66
realizado entre los 30 procedimientos disponibles. Por lo tanto, 1 de los egresos debe contener
2 angioplastias.

In [21]:
conteo_de_angio = np.where(control_angioplastias[vars_procedimientos.keys()] == "00.66", 1, 0)

In [22]:
display(conteo_de_angio.sum(axis=0))

array([818,   1,   2,   1,   0,   2,   1,   3,   0,   0,   0,   1,   0,
         0,   0,   0,   1,   1,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

En el resultado anterior se indica la frecuencia de angioplastias por posición de procedimiento.
Estos indican que 818 angioplastias se realizaron en el PROCEDIMIENTO1. Además, también se realizaron
angioplastias en los procedimientos 2, 3, 4, 6, 7, 8, 12, 17 y 18.

In [23]:
paciente_con_dos_angios = pd.DataFrame(conteo_de_angio)[
    pd.DataFrame(conteo_de_angio).sum(axis=1) == 2
].index

control_angioplastias.iloc[paciente_con_dos_angios][list(vars_procedimientos.keys())[:20]]

,PROCEDIMIENTO1,PROCEDIMIENTO2,PROCEDIMIENTO3,PROCEDIMIENTO4,PROCEDIMIENTO5,PROCEDIMIENTO6,PROCEDIMIENTO7,PROCEDIMIENTO8,PROCEDIMIENTO9,PROCEDIMIENTO10,PROCEDIMIENTO11,PROCEDIMIENTO12,PROCEDIMIENTO13,PROCEDIMIENTO14,PROCEDIMIENTO15,PROCEDIMIENTO16,PROCEDIMIENTO17,PROCEDIMIENTO18,PROCEDIMIENTO19,PROCEDIMIENTO20
910192,00.66,00.41,00.47,36.07,36.09,37.61,88.55,37.22,96.71,96.04,99.60,99.63,38.91,57.94,96.08,38.93,00.66,99.05,99.21,99.26


Efectivamente, existe un paciente que se realizó 2 angioplastias durante su egreso. Su primera
angioplastia fue realizada como procedimiento 1, y luego se realizó otra como procedimiento 17.